In [ ]:
from Pipeline.ccb.dsjson.processor import Processor
from Pipeline.ccb.dsjson.predictions import Predictor
from Pipeline import azure_blob_logs
from Pipeline.utils import Mapper
from Pipeline.dataflow import MultilineFiles, PickleFiles, CsvFiles

from pathlib import Path

In [ ]:
local_data_root = Path('/Users/alextaim/data/ccb')

raw_folder = local_data_root.joinpath('raw')
slim_folder = local_data_root.joinpath('slim')
sample_folder = local_data_root.joinpath('sample')
predict_folder = local_data_root.joinpath('predict.pickle')
baseline_predict_folder = local_data_root.joinpath('baseline.predict')
baseline_estimate_folder = local_data_root.joinpath('baseline.estimate')

In [ ]:
files = ['/Users/alextaim/data/ccb/raw/01.json',
    '/Users/alextaim/data/ccb/raw/02.json']

raw = MultilineFiles(files)

# Generate predictions

In [ ]:
from VwPipeline import Loggers, Handlers
from VwPipeline.VwCache import VwCache
from VwPipeline.Vw import Vw
from VwPipeline.VwOpts import dimension, product
import pandas as pd

#your vw path
vw_path = r'vw'

cache = VwCache(r'/Users/alextaim/data/.vw_cache')
vw = Vw(
    vw_path,
    cache,
    handlers=[Handlers.WidgetHandler()],
    )

opts = pd.DataFrame(product(
    dimension('#base', ['--ccb_explore_adf --dsjson   -P 1 --preserve_performance_counters --save_resume']),
    dimension('#learning', ['--coin']),
))
preds = vw.train(files, opts, ['-p'])
prediction_files = {preds.iloc[0]['!Job'].name: preds.iloc[0]['!Outputs']['-p']}
prediction_files

# Generate slim dsjson

In [ ]:
from Pipeline.ccb.dsjson import processor

In [ ]:
from Pipeline.ccb.dsjson.processor import Processor

processor = Processor()
slim = MultilineFiles().init(raw.process(processor.process, path_gen=Mapper(raw_folder, slim_folder), process=True))

# Sample

In [ ]:

from Pipeline.filters import UniformSampler
from Pipeline.ccb.dsjson.processor import Processor

sampler = UniformSampler(0.5)
processor = Processor(filters=[lambda l: sampler.do(l)])
sample = MultilineFiles().init(raw.process(processor.process, path_gen=Mapper(raw_folder, sample_folder), process=True))

# Predict

In [ ]:
from Pipeline.ccb.dsjson.predictions import Predictor

predictor = Predictor(filters=[lambda l: True])
baseline_preds = PickleFiles().init(raw.process(predictor.predict_df, path_gen=Mapper(raw_folder, baseline_predict_folder), process=True))

# Preestimate

In [ ]:
from Pipeline.estimators import Estimator, evaluate
import Pipeline.ccb.estimators
import json

pipeline = FilesPipeline()
estimator = Estimator(factory = Pipeline.ccb.estimators.create, estimators = {'baseline1_old': ['ccb|snips|0'], 'random': ['ccb|snips|0']}, online_estimator = 'ccb|ips|0', window='1min')
baseline_preestimates = PickleFiles().init(baseline_preds.process(estimator.preestimate_df, path_gen=Mapper(baseline_predict_folder, baseline_estimate_folder), process=True))


# Evaluate

In [ ]:
evaluate(baseline_preestimates.open().resample('2min').sum())


# Evaluate predictions from vw

In [ ]:
from Pipeline.ccb.vw import predictions
cfe_folder = local_data_root.joinpath('cfe.pickle')
policy_name = '--coin'
vw_pred_openers = list(map(lambda p: (lambda f: predictions.lines_2_predictions(open(p), pd.read_pickle(f), policy_name)), prediction_files[policy_name]))

pipeline = FilesPipeline()
estimator = Estimator(factory = Pipeline.ccb.estimators.create, estimators = {'cfe': ['ccb|snips|0'] })
preestimates = pipeline.df_pickle_2_df_pickle(cfe, lambda objects: estimator.preestimate_df(objects, '1min'), path_gen=Mapper(predict_folder, cfe_folder), process=True, openers=vw_pred_openers)


In [ ]:
from Pipeline.ccb.vw import predictions
estimator = Estimator(factory = Pipeline.ccb.estimators.create, estimators = {'--coin': ['ccb|snips|0'] })
p = predictions.lines_2_predictions_df(open(prediction_files['--coin'][0]), pd.read_pickle(cfe[0]), '--coin')
p

In [ ]:
from Pipeline.ccb.vw import predictions
vw_pred_openers = list(map(lambda p: (lambda f: predictions.lines_2_predictions(open(p), open(f), 'cfe')), prediction_files))

pipeline = FilesPipeline()
estimator = Estimator(factory = Pipeline.ccb.estimators.create, estimators = {'cfe': ['ccb|snips|0'] })
preestimates = pipeline.ndjson_2_csv(cfe, lambda objects: estimator.preestimate(objects, '1min'), path_gen=lambda p: p.replace('predict', 'cfe'), process=True, openers=vw_pred_openers)


In [ ]:
baseline_preestimates = pd.concat([estimator.read_preestimate(p) for p in preestimates])
cfe_stats = evaluate(baseline_preestimates.resample('1min').sum())
cfe_stats

# Merge evaluations

In [ ]:
all_stats = baseline_stats.join(cfe_stats)